In [13]:
%load_ext autoreload
%autoreload 2

import sys
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np
import matplotlib.pyplot as plt

LOG.propagate = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
LOG.debug("debug")
LOG.info("info")
LOG.warning("warning")
LOG.error("error")
LOG.critical("critical")

2024-03-07 05:05:08,721 | INFO     |: info
2024-03-07 05:05:08,722 | WARNING  |: warning
2024-03-07 05:05:08,723 | ERROR    |: error
2024-03-07 05:05:08,724 | CRITICAL |: critical


In [15]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2024-03-07 05:05:14,977 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:89:c1:6d:24:4b
2024-03-07 05:05:19,792 | INFO     |: Connected to c0:89:c1:6d:24:4b


In [16]:
ble.send_command(CMD.ECHO, "HiHello")
output1 = ble.receive_string(ble.uuid['RX_STRING'])
print(output1)

Robot says -> HiHello :)


In [17]:
ble.send_command(CMD.PING, "")

In [18]:
raw_message=[]
#add more arrays if needed
num = []
time = [] #in seconds
xAcc = [] 
yAcc = []
zAcc = []
xGyr = [] 
yGyr = []
zGyr = []
def notification_handler(uuid, byte_array):
    global time,xAcc,yAcc,zAcc,xGyr,yGyr,zGyr
    raw_message.append(byte_array)
    # Extracts the time portion in seconds from the byte array
    message = ble.bytearray_to_string(byte_array)
    readings = message.split('|')
    num.clear()
    for reading in readings:
        label, value = reading.split(':')
        num.append(value)
    #Add more arrays if needed
    time.append(float(num[0])/1000)
    xAcc.append(num[1])
    yAcc.append(num[2])
    zAcc.append(num[3])
    xGyr.append(num[4])
    yGyr.append(num[5])
    zGyr.append(num[6])
    #print(f"Time: {float(num[0])/1000}s; xAcc: {num[1]}; yAcc: {num[2]}; xAcc: {num[1]}; xAcc: {num[1]}; xAcc: {num[1]}; xAcc: {num[1]}")

In [19]:
ble.start_notify(ble.uuid['RX_STRING'], notification_handler)

In [20]:
ble.send_command(CMD.GET_IMU_DATA, "")

In [10]:
print(f"It takes: {round(time[-1]-time[0],3)}s to collect data.")
print(f"Sample rate is about {round(len(time)/(time[-1]-time[0]))} samples/s.")
for i in range(len(time)):
    print(f"Time: {time[i]}s; xAcc: {xAcc[i]}; yAcc: {yAcc[i]}; zAcc: {zAcc[i]}; xGyr: {xGyr[i]}; yGyr: {yGyr[i]}; zGyr: {zGyr[i]}")

It takes: 0.271s to collect data.
Sample rate is about 369 samples/s.
Time: 74.2s; xAcc: 55.175; yAcc: 4.394; zAcc: 1050.781; xGyr: -1.435; yGyr: 0.687; zGyr: -2.503
Time: 74.206s; xAcc: 44.433; yAcc: -13.183; zAcc: 1045.410; xGyr: 0.435; yGyr: 1.557; zGyr: 0.290
Time: 74.208s; xAcc: 54.199; yAcc: -3.417; zAcc: 1045.898; xGyr: -1.320; yGyr: 1.229; zGyr: 0.664
Time: 74.212s; xAcc: 69.335; yAcc: 9.765; zAcc: 1048.339; xGyr: 0.549; yGyr: -3.213; zGyr: -2.76
Time: 74.214s; xAcc: 60.58; yAcc: 7.324; zAcc: 1044.921; xGyr: -2.984; yGyr: 0.748; zGyr: 0.381
Time: 74.216s; xAcc: 63.964; yAcc: -2.441; zAcc: 1040.527; xGyr: -1.312; yGyr: 1.610; zGyr: 0.38
Time: 74.221s; xAcc: 72.265; yAcc: 2.441; zAcc: 1046.386; xGyr: 0.442; yGyr: 2.793; zGyr: 0.641
Time: 74.223s; xAcc: 51.757; yAcc: 12.695; zAcc: 1055.664; xGyr: -1.984; yGyr: 5.351; zGyr: -2.435
Time: 74.225s; xAcc: 58.105; yAcc: 6.835; zAcc: 1050.781; xGyr: 0.221; yGyr: 0.274; zGyr: 0.259
Time: 74.228s; xAcc: 61.523; yAcc: -6.835; zAcc: 1037.597

In [12]:
raw_message

[bytearray(b'Time:74200.0|ACC-X:55.175|ACC-Y:4.394|ACC-Z:1050.781|Gyr-X:-1.435|Gyr-Y:0.687|Gyr-Z:-2.503'),
 bytearray(b'Time:74206.0|ACC-X:44.433|ACC-Y:-13.183|ACC-Z:1045.410|Gyr-X:0.435|Gyr-Y:1.557|Gyr-Z:0.290'),
 bytearray(b'Time:74208.0|ACC-X:54.199|ACC-Y:-3.417|ACC-Z:1045.898|Gyr-X:-1.320|Gyr-Y:1.229|Gyr-Z:0.664'),
 bytearray(b'Time:74212.0|ACC-X:69.335|ACC-Y:9.765|ACC-Z:1048.339|Gyr-X:0.549|Gyr-Y:-3.213|Gyr-Z:-2.76'),
 bytearray(b'Time:74214.0|ACC-X:60.58|ACC-Y:7.324|ACC-Z:1044.921|Gyr-X:-2.984|Gyr-Y:0.748|Gyr-Z:0.381'),
 bytearray(b'Time:74216.0|ACC-X:63.964|ACC-Y:-2.441|ACC-Z:1040.527|Gyr-X:-1.312|Gyr-Y:1.610|Gyr-Z:0.38'),
 bytearray(b'Time:74221.0|ACC-X:72.265|ACC-Y:2.441|ACC-Z:1046.386|Gyr-X:0.442|Gyr-Y:2.793|Gyr-Z:0.641'),
 bytearray(b'Time:74223.0|ACC-X:51.757|ACC-Y:12.695|ACC-Z:1055.664|Gyr-X:-1.984|Gyr-Y:5.351|Gyr-Z:-2.435'),
 bytearray(b'Time:74225.0|ACC-X:58.105|ACC-Y:6.835|ACC-Z:1050.781|Gyr-X:0.221|Gyr-Y:0.274|Gyr-Z:0.259'),
 bytearray(b'Time:74228.0|ACC-X:61.523|ACC-Y

2024-03-07 05:04:50,515 | INFO     |: Disconnected from C0:89:C1:6D:24:4B


In [11]:
ble.stop_notify(ble.uuid['RX_STRING'])

In [11]:
# Disconnect
ble.disconnect()

2024-02-21 13:03:53,859 | INFO     |: Disconnected from C0:89:C1:6D:24:4B
